In [ ]:
import sys
print(sys.version)


In [ ]:
import sys
print(sys.executable)


In [ ]:
!pip install -q mediapipe

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh



In [ ]:
import os

In [ ]:
# For static images:

# Read all images from folder
image_dir = "C:\\Users\\prave\\OneDrive\\Desktop\\Git\\Face-Animator\\test"
IMAGE_FILES = [os.path.join(image_dir, file)
               for file in os.listdir(image_dir)
               if file.endswith(('.jpg', '.png', '.jpeg'))]

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    # Convert the BGR image to RGB before processing.
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Print and draw face mesh landmarks on the image.
    if not results.multi_face_landmarks:
      continue
    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:
      print('face_landmarks:', face_landmarks)
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_TESSELATION,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_tesselation_style())
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_CONTOURS,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_contours_style())
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_iris_connections_style())
    cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)


In [ ]:
from ffpyplayer.player import MediaPlayer
import cv2

def PlayVideo(video_path):
    video = cv2.VideoCapture(video_path)
    player = MediaPlayer(video_path)

    while True:
        grabbed, frame = video.read()
        audio_frame, val = player.get_frame()

        if not grabbed:
            print("End of video")
            break

        cv2.imshow("Video", frame)

        if val != 'eof' and audio_frame is not None:
            img, t = audio_frame

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()

PlayVideo("C:\\Users\\prave\\OneDrive\\Desktop\\Git\\Face-Animator\\test\\exp_test.mp4")


In [ ]:
import time
from ffpyplayer.player import MediaPlayer

In [ ]:


# Drawing specifications
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Video path
video_path = "C:\\Users\\prave\\OneDrive\\Desktop\\Git\\Face-Animator\\test\\exp_test.mp4"
cap = cv2.VideoCapture(video_path)
player = MediaPlayer(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error opening video file")
    exit()

try:
    with mp_face_mesh.FaceMesh(
        max_num_faces=3,
        refine_landmarks=True,
        min_detection_confidence=0.8,
        min_tracking_confidence=0.8) as face_mesh:

        while cap.isOpened():
            start_time = time.time()  # Start inference timer

            success, image = cap.read()
            if not success:
                print("Video stream ended.")
                break

            image.flags.writeable = False
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(image_rgb)

            # Retrieve and play audio frame
            audio_frame, val = player.get_frame()
            if val == 'eof':
                print("Audio stream ended.")
                break

            image.flags.writeable = True

            # Draw face landmarks
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    mp_drawing.draw_landmarks(
                        image=image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
                    mp_drawing.draw_landmarks(
                        image=image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_CONTOURS,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
                    mp_drawing.draw_landmarks(
                        image=image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_IRISES,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style())

            # Calculate and display inference time
            end_time = time.time()
            inference_time_ms = (end_time - start_time) * 1000
            cv2.putText(image, f"Inference time: {inference_time_ms:.2f} ms",
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Display the output frame
            cv2.imshow('MediaPipe Face Mesh', image)

            # Exit on pressing 'q'
            if cv2.waitKey(25) & 0xFF == ord('q'):
                print("Exiting on user command.")
                break

finally:
    # Always release resources and close windows
    cap.release()
    player.close_player()
    cv2.destroyAllWindows()
    print("Resources released. Program ended.")


In [ ]:
import numpy as np
import time

Face alignment using the keypoints of the eyes,lips,and nose.

In [ ]:


mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Drawing specifications
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Video path
video_path = "C:\\Users\\prave\\OneDrive\\Desktop\\Git\\Face-Animator\\test\\exp_test.mp4"
cap = cv2.VideoCapture(video_path)

# Standard face landmark indices for alignment
LEFT_EYE_IDX = 33   # Approximate landmark index for left eye
RIGHT_EYE_IDX = 263 # Approximate landmark index for right eye
NOSE_IDX = 1        # Nose tip

# Desired output image size and landmark positions (canonical template)
output_size = (512, 512)
# Slight zoom out by reducing eye distance
dst_pts = np.float32([
    [230, 240],  # Left eye moved further right
    [282, 240],  # Right eye moved further left
    [256, 340]   # Nose tip moved slightly higher
])
# Check if video opened successfully
if not cap.isOpened():
    print("Error opening video file")
    exit()

try:
    with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.8,
        min_tracking_confidence=0.8) as face_mesh:

        while cap.isOpened():
            start_time = time.time()

            success, image = cap.read()
            if not success:
                print("Video stream ended.")
                break

            image.flags.writeable = False
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(image_rgb)
            image.flags.writeable = True

            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # Extract landmark coordinates for alignment
                    h, w, _ = image.shape
                    left_eye = face_landmarks.landmark[LEFT_EYE_IDX]
                    right_eye = face_landmarks.landmark[RIGHT_EYE_IDX]
                    nose = face_landmarks.landmark[NOSE_IDX]

                    src_pts = np.float32([
                        [left_eye.x * w, left_eye.y * h],
                        [right_eye.x * w, right_eye.y * h],
                        [nose.x * w, nose.y * h]
                    ])

                    # Compute affine transform for alignment
                    M = cv2.getAffineTransform(src_pts, dst_pts)
                    aligned_image = cv2.warpAffine(image, M, output_size)

                    # Draw landmarks on aligned image
                    mp_drawing.draw_landmarks(
                        image=aligned_image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
                    mp_drawing.draw_landmarks(
                        image=aligned_image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_CONTOURS,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
                    mp_drawing.draw_landmarks(
                        image=aligned_image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_IRISES,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style())

                    # Calculate and display inference time
                    end_time = time.time()
                    inference_time_ms = (end_time - start_time) * 1000
                    cv2.putText(aligned_image, f"Inference time: {inference_time_ms:.2f} ms",
                                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                    # Display aligned output
                    cv2.imshow('Aligned Face Mesh', aligned_image)

            # Exit on pressing 'q'
            if cv2.waitKey(25) & 0xFF == ord('q'):
                print("Exiting on user command.")
                break

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("Resources released. Program ended.")


Face Alignment ratios:

In [ ]:
# Drawing specifications
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Video path
video_path = "C:\\Users\\prave\\OneDrive\\Desktop\\Git\\Face-Animator\\test\\exp_test.mp4"
cap = cv2.VideoCapture(video_path)

# Standard face landmark indices for alignment
LEFT_EYE_IDX = 33   # Approximate left eye
RIGHT_EYE_IDX = 263 # Approximate right eye
NOSE_IDX = 1        # Nose tip

# Check if video opened successfully
if not cap.isOpened():
    print("Error opening video file")
    exit()

try:
    with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.8,
        min_tracking_confidence=0.8) as face_mesh:

        while cap.isOpened():
            start_time = time.time()

            success, image = cap.read()
            if not success:
                print("Video stream ended.")
                break

            image.flags.writeable = False
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(image_rgb)
            image.flags.writeable = True

            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # Extract original frame size
                    h, w, _ = image.shape

                    # Scaling factor (e.g. 1.5x larger)
                    scale = 1.5
                    output_width = int(w * scale)
                    output_height = int(h * scale)
                    output_size = (output_width, output_height)

                    # Destination landmark positions relative to output size
                    dst_pts = np.float32([
                        [output_width * 0.35, output_height * 0.4],  # Left eye
                        [output_width * 0.65, output_height * 0.4],  # Right eye
                        [output_width * 0.5, output_height * 0.6]    # Nose tip
                    ])

                    # Extract landmark coordinates for alignment
                    left_eye = face_landmarks.landmark[LEFT_EYE_IDX]
                    right_eye = face_landmarks.landmark[RIGHT_EYE_IDX]
                    nose = face_landmarks.landmark[NOSE_IDX]

                    src_pts = np.float32([
                        [left_eye.x * w, left_eye.y * h],
                        [right_eye.x * w, right_eye.y * h],
                        [nose.x * w, nose.y * h]
                    ])

                    # Compute affine transform
                    M = cv2.getAffineTransform(src_pts, dst_pts)
                    aligned_image = cv2.warpAffine(image, M, output_size)

                    # Draw landmarks on aligned image
                    mp_drawing.draw_landmarks(
                        image=aligned_image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
                    mp_drawing.draw_landmarks(
                        image=aligned_image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_CONTOURS,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
                    mp_drawing.draw_landmarks(
                        image=aligned_image,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_IRISES,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style())

                    # Calculate and display inference time
                    end_time = time.time()
                    inference_time_ms = (end_time - start_time) * 1000
                    cv2.putText(aligned_image, f"Inference time: {inference_time_ms:.2f} ms",
                                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                    # Display aligned output
                    cv2.imshow('Aligned Face Mesh', aligned_image)

            # Exit on pressing 'q'
            if cv2.waitKey(25) & 0xFF == ord('q'):
                print("Exiting on user command.")
                break

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("Resources released. Program ended.")